In [0]:
"""
We use following lines because we are running on Google Colab
If you are running notebook on a local computer, you don't need this cell
"""
from google.colab import drive
drive.mount('/content/gdrive')
import os
os.chdir('/content/gdrive/My Drive/finch/tensorflow1/free_chat/chinese/main')

In [0]:
import tensorflow as tf

import numpy as np
import pprint
import logging
import time
import os
import random

from pathlib import Path

print("TensorFlow Version", tf.__version__)
print('GPU Enabled:', tf.test.is_gpu_available())

TensorFlow Version 1.14.0
GPU Enabled: True


In [0]:
# stream data from text files
def data_generator(f_path, params):
  char2idx = params['char2idx']
  with open(f_path) as f:
    print('Reading', f_path)
    for line in f:
      line = line.rstrip()
      sp = line.split('|')
      if len(sp) == 2:
        source, target = sp
      else:
        continue
      source = [char2idx.get(c, len(char2idx)) for c in list(source)]
      target = [char2idx.get(c, len(char2idx)) for c in list(target)]
      target_in = [1] + target
      target_out = target + [2]
      if len(source) > 0:
        yield (source, (target_in, target_out))

In [0]:
def dataset(is_training, params):
  _shapes = ([None], ([None], [None]))
  _types = (tf.int32, (tf.int32, tf.int32))
  _pads = (0, (0, 0))
  
  if is_training:
    ds = tf.data.Dataset.from_generator(
      lambda: data_generator(params['train_path'], params),
      output_shapes = _shapes,
      output_types = _types,)
    ds = ds.shuffle(params['buffer_size'])
    ds = ds.padded_batch(params['batch_size'], _shapes, _pads)
    ds = ds.prefetch(tf.data.experimental.AUTOTUNE)
  else:
    ds = tf.data.Dataset.from_generator(
      lambda: data_generator(params['test_path'], params),
      output_shapes = _shapes,
      output_types = _types,)
    ds = ds.padded_batch(params['batch_size'], _shapes, _pads)
    ds = ds.prefetch(tf.data.experimental.AUTOTUNE)
  
  return ds

In [0]:
def clip_grads(loss):
    variables = tf.trainable_variables()
    pprint.pprint(variables)
    grads = tf.gradients(loss, variables)
    clipped_grads, _ = tf.clip_by_global_norm(grads, params['clip_norm'])
    return zip(clipped_grads, variables)


def rnn_cell():
    def cell_fn():
        cell = tf.nn.rnn_cell.GRUCell(params['rnn_units'],
                                      kernel_initializer=tf.orthogonal_initializer())
        return cell
    if params['dec_layers'] > 1:
      return tf.nn.rnn_cell.MultiRNNCell([cell_fn() for _ in range(params['dec_layers'])])
    else:
      return cell_fn()

  
def dec_cell(enc_out, enc_seq_len):
    attn1 = tf.contrib.seq2seq.BahdanauAttention(
        num_units = params['rnn_units'],
        memory = enc_out,
        memory_sequence_length = enc_seq_len,
        normalize=False)
    
    attn2 = tf.contrib.seq2seq.LuongAttention(
        num_units = params['rnn_units'],
        memory = enc_out,
        memory_sequence_length = enc_seq_len,
        scale=False)
    
    return tf.contrib.seq2seq.AttentionWrapper(
        cell = rnn_cell(),
        attention_mechanism = (attn1, attn2),
        attention_layer_size = (params['rnn_units'], params['rnn_units']))

In [0]:
class TiedDense(tf.layers.Layer):
  def __init__(self, tied_embed, out_dim):
    super().__init__()
    self.tied_embed = tied_embed
    self.out_dim = out_dim
  
  def build(self, input_shape):
    self.bias = self.add_weight(name='bias',
                                shape=[self.out_dim],
                                trainable=True)
    self.proj_W = self.add_weight(name='proj_W',
                                  shape=[2*params['rnn_units'], params['rnn_units']],
                                  trainable=True)
    self.proj_b = self.add_weight(name='proj_b',
                                  shape=[params['rnn_units']],
                                  trainable=True)
    super().build(input_shape)
  
  def call(self, inputs):
    x = tf.nn.elu(tf.nn.bias_add(tf.matmul(inputs, self.proj_W), self.proj_b))
    x = tf.matmul(x, self.tied_embed, transpose_b=True)
    x = tf.nn.bias_add(x, self.bias)
    return x
  
  def compute_output_shape(self, input_shape):
    return input_shape[:-1].concatenate(self.out_dim)

In [0]:
def forward(words, labels, mode):
    words_len = tf.count_nonzero(words, 1, dtype=tf.int32)
    
    is_training = (mode == tf.estimator.ModeKeys.TRAIN)
    batch_sz = tf.shape(words)[0]
    
    
    with tf.variable_scope('Embedding'):
        embedding = tf.get_variable('embedding', [len(params['char2idx'])+1, params['rnn_units']], tf.float32)
        x = tf.nn.embedding_lookup(embedding, words)
        x = tf.layers.dropout(x, params['dropout_rate'], training=is_training)
    
    
    with tf.variable_scope('Encoder'):
        cell_fw = tf.nn.rnn_cell.GRUCell(params['rnn_units'],
                                         kernel_initializer=tf.orthogonal_initializer())
        cell_bw = tf.nn.rnn_cell.GRUCell(params['rnn_units'],
                                         kernel_initializer=tf.orthogonal_initializer())
        (o_fw, o_bw), (s_fw, s_bw) = tf.nn.bidirectional_dynamic_rnn(
            cell_fw, cell_bw, x, sequence_length=words_len, dtype=tf.float32)
        enc_out = tf.concat((o_fw, o_bw), -1)
        enc_state = tf.layers.dense(tf.concat((s_fw, s_bw), -1), params['rnn_units'], tf.nn.elu)
        if params['dec_layers'] > 1:
          enc_state = tuple(params['dec_layers'] * [enc_state])
    
    
    with tf.variable_scope('Decoder'):
        output_proj = TiedDense(embedding, len(params['char2idx'])+1)
        
        if is_training or (mode == tf.estimator.ModeKeys.EVAL):
            dec_inputs, dec_outputs = labels
            dec_seq_len = tf.count_nonzero(dec_inputs, 1, dtype=tf.int32)
            dec_inputs = tf.nn.embedding_lookup(embedding, dec_inputs)
            dec_inputs = tf.layers.dropout(dec_inputs, params['dropout_rate'], training=is_training)
            cell = dec_cell(enc_out, words_len)
            
            init_state = cell.zero_state(batch_sz, tf.float32).clone(
                cell_state=enc_state)
            
            helper = tf.contrib.seq2seq.TrainingHelper(
                inputs = dec_inputs,
                sequence_length = dec_seq_len,)
            decoder = tf.contrib.seq2seq.BasicDecoder(
                cell = cell,
                helper = helper,
                initial_state = init_state,
                output_layer = output_proj)
            decoder_output, _, _ = tf.contrib.seq2seq.dynamic_decode(
                decoder = decoder,
                maximum_iterations = tf.reduce_max(dec_seq_len))
            
            return decoder_output.rnn_output
        else:
            enc_out_t = tf.contrib.seq2seq.tile_batch(enc_out, params['beam_width'])
            enc_state_t = tf.contrib.seq2seq.tile_batch(enc_state, params['beam_width'])
            enc_seq_len_t = tf.contrib.seq2seq.tile_batch(words_len, params['beam_width'])
            
            cell = dec_cell(enc_out_t, enc_seq_len_t)
            
            init_state = cell.zero_state(batch_sz*params['beam_width'], tf.float32).clone(
                cell_state=enc_state_t)
            
            decoder = tf.contrib.seq2seq.BeamSearchDecoder(
                cell = cell,
                embedding = embedding,
                start_tokens = tf.tile(tf.constant([1], tf.int32), [batch_sz]),
                end_token = 2,
                initial_state = init_state,
                beam_width = params['beam_width'],
                output_layer = output_proj,
                length_penalty_weight=1.0,)
            decoder_output, _, _ = tf.contrib.seq2seq.dynamic_decode(
                decoder = decoder,
                maximum_iterations = 30,)
            
            return decoder_output.predicted_ids[:, :, 0]

In [0]:
def model_fn(features, labels, mode, params):
    logits_or_ids = forward(features, labels, mode)
    
    if mode == tf.estimator.ModeKeys.PREDICT:
        return tf.estimator.EstimatorSpec(mode, predictions=logits_or_ids)
    
    dec_inputs, dec_outputs = labels
    loss_op = tf.contrib.seq2seq.sequence_loss(logits = logits_or_ids,
                                               targets = dec_outputs,
                                               weights = tf.to_float(tf.sign(dec_outputs)))  
      
    if mode == tf.estimator.ModeKeys.TRAIN:
        global_step=tf.train.get_or_create_global_step()
        
        decay_lr = tf.train.exponential_decay(
            params['lr'], global_step, 1000, .96)
        
        train_op = tf.train.AdamOptimizer(decay_lr).apply_gradients(
            clip_grads(loss_op), global_step=global_step)
        
        hook = tf.train.LoggingTensorHook({'lr': decay_lr}, every_n_iter=100)
        
        return tf.estimator.EstimatorSpec(
            mode=mode, loss=loss_op, train_op=train_op, training_hooks=[hook],)
      
    if mode == tf.estimator.ModeKeys.EVAL:
      return tf.estimator.EstimatorSpec(mode=mode, loss=loss_op)

In [0]:
def get_vocab(f_path):
  word2idx = {}
  with open(f_path) as f:
    for i, line in enumerate(f):
      line = line.rstrip('\n')
      word2idx[line] = i
  return word2idx

In [0]:
def pad(test_strs):
  max_len = max([len(test_str) for test_str in test_strs])
  for test_str in test_strs:
    if len(test_str) < max_len:
      test_str += ['<pad>'] * (max_len - len(test_str))


def minimal_test(estimator):
  test_strs = [
    '你是谁啊',
    '陪我玩',
    '二加五等于几',
    '讲个笑话',
    '今天天气如何',
  ]
  test_strs = [list(test_str) for test_str in test_strs]
  pad(test_strs)
  test_arrs = [[params['char2idx'].get(c, len(params['char2idx'])) for c in test_str] for test_str in test_strs]
  predicted = list(estimator.predict(tf.estimator.inputs.numpy_input_fn(
    x = np.array(test_arrs), shuffle = False)))
  print('-'*12)
  print('minimal test')
  for i, test_str in enumerate(test_strs):
    print('utterance:', ' '.join(test_str))
    sent = ' '.join([params['idx2char'].get(idx, len(params['idx2char'])) for idx in predicted[i]])
    sent = sent.replace('<end>', '')
    print('predicted:', sent)
    print()
  print('-'*12)

In [0]:
def is_increasing(history: list) -> bool:
  history = history[-(params['num_patience']+1):]
  for i in range(1, len(history)):
    if history[i-1] >= history[i]:
      return False
  return True  

In [0]:
params = {
    'model_dir': '../model/gru_seq2seq',
    'log_path': '../log/gru_seq2seq.txt',
    'train_path': '../data/train.txt',
    'test_path': '../data/test.txt',
    'vocab_path': '../vocab/char.txt',
    'model_path': '../model/',
    'dropout_rate': 0.2,
    'rnn_units': 512,
    'dec_layers': 1,
    'beam_width': 10,
    'lr': 4e-4,
    'clip_norm': 5.,
    'buffer_size': 400000,
    'batch_size': 32,
    'num_patience': 5,
}

In [0]:
params['char2idx'] = get_vocab(params['vocab_path'])
params['idx2char'] = {idx: char for char, idx in params['char2idx'].items()}

In [0]:
# Create directory if not exist
Path(os.path.dirname(params['log_path'])).mkdir(exist_ok=True)
Path(params['model_dir']).mkdir(exist_ok=True, parents=True)

# Logging
logger = logging.getLogger('tensorflow')
logger.setLevel(logging.INFO)
fh = logging.FileHandler(params['log_path'])
logger.addHandler(fh)

# Create an estimator
eval_steps = params['buffer_size'] // params['batch_size'] // 5
estimator = tf.estimator.Estimator(
  model_fn=model_fn,
  model_dir=params['model_dir'],
  config=tf.estimator.RunConfig(save_checkpoints_steps=eval_steps),
  params=params)

best_loss = .0
history_loss = []

while True:
  estimator.train(input_fn=lambda: dataset(is_training=True, params=params), steps=eval_steps)

  minimal_test(estimator)

  loss = estimator.evaluate(input_fn=lambda: dataset(is_training=False, params=params))['loss']

  history_loss.append(loss)

  if loss < best_loss:
    best_loss = loss
  logger.info("Best Loss: {:.3f}".format(best_loss))

  if len(history_loss) > params['num_patience'] and is_increasing(history_loss):
    logger.info("Testing Loss not improved over {} epochs, Early Stop".format(params['num_patience']))
    break

I0823 00:48:39.220615 140559935440768 estimator.py:209] Using config: {'_model_dir': '../model/gru_seq2seq', '_tf_random_seed': None, '_save_summary_steps': 100, '_save_checkpoints_steps': 2500, '_save_checkpoints_secs': None, '_session_config': allow_soft_placement: true
graph_options {
  rewrite_options {
    meta_optimizer_iterations: ONE
  }
}
, '_keep_checkpoint_max': 5, '_keep_checkpoint_every_n_hours': 10000, '_log_step_count_steps': 100, '_train_distribute': None, '_device_fn': None, '_protocol': None, '_eval_distribute': None, '_experimental_distribute': None, '_experimental_max_worker_delay_secs': None, '_service': None, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x7fd6520bda58>, '_task_type': 'worker', '_task_id': 0, '_global_id_in_cluster': 0, '_master': '', '_evaluation_master': '', '_is_chief': True, '_num_ps_replicas': 0, '_num_worker_replicas': 1}
W0823 00:48:39.615683 140559935440768 deprecation.py:323] From /usr/local/lib/python3.6/d

[<tf.Variable 'Embedding/embedding:0' shape=(4718, 512) dtype=float32_ref>,
 <tf.Variable 'Encoder/bidirectional_rnn/fw/gru_cell/gates/kernel:0' shape=(1024, 1024) dtype=float32_ref>,
 <tf.Variable 'Encoder/bidirectional_rnn/fw/gru_cell/gates/bias:0' shape=(1024,) dtype=float32_ref>,
 <tf.Variable 'Encoder/bidirectional_rnn/fw/gru_cell/candidate/kernel:0' shape=(1024, 512) dtype=float32_ref>,
 <tf.Variable 'Encoder/bidirectional_rnn/fw/gru_cell/candidate/bias:0' shape=(512,) dtype=float32_ref>,
 <tf.Variable 'Encoder/bidirectional_rnn/bw/gru_cell/gates/kernel:0' shape=(1024, 1024) dtype=float32_ref>,
 <tf.Variable 'Encoder/bidirectional_rnn/bw/gru_cell/gates/bias:0' shape=(1024,) dtype=float32_ref>,
 <tf.Variable 'Encoder/bidirectional_rnn/bw/gru_cell/candidate/kernel:0' shape=(1024, 512) dtype=float32_ref>,
 <tf.Variable 'Encoder/bidirectional_rnn/bw/gru_cell/candidate/bias:0' shape=(512,) dtype=float32_ref>,
 <tf.Variable 'Encoder/dense/kernel:0' shape=(1024, 512) dtype=float32_ref>,

I0823 00:48:47.704582 140559935440768 estimator.py:1147] Done calling model_fn.
I0823 00:48:47.708714 140559935440768 basic_session_run_hooks.py:541] Create CheckpointSaverHook.
I0823 00:48:48.646365 140559935440768 monitored_session.py:240] Graph was finalized.
W0823 00:48:48.987824 140559935440768 deprecation.py:323] From /usr/local/lib/python3.6/dist-packages/tensorflow/python/training/saver.py:1276: checkpoint_exists (from tensorflow.python.training.checkpoint_management) is deprecated and will be removed in a future version.
Instructions for updating:
Use standard file APIs to check for files with this prefix.
I0823 00:48:49.000391 140559935440768 saver.py:1280] Restoring parameters from ../model/gru_seq2seq/model.ckpt-10000
W0823 00:48:54.488085 140559935440768 deprecation.py:323] From /usr/local/lib/python3.6/dist-packages/tensorflow/python/training/saver.py:1066: get_checkpoint_mtimes (from tensorflow.python.training.checkpoint_management) is deprecated and will be removed in a

Reading ../data/train.txt


I0823 00:50:21.766146 140559935440768 basic_session_run_hooks.py:262] loss = 3.5489733, step = 10000
I0823 00:50:21.768124 140559935440768 basic_session_run_hooks.py:262] lr = 0.000265933
I0823 00:51:13.073870 140559935440768 basic_session_run_hooks.py:692] global_step/sec: 1.949
I0823 00:51:13.080963 140559935440768 basic_session_run_hooks.py:260] loss = 2.4353392, step = 10100 (51.315 sec)
I0823 00:51:13.084181 140559935440768 basic_session_run_hooks.py:260] lr = 0.00026484957 (51.316 sec)
I0823 00:52:11.899215 140559935440768 basic_session_run_hooks.py:692] global_step/sec: 1.69995
I0823 00:52:11.904767 140559935440768 basic_session_run_hooks.py:260] loss = 3.4213533, step = 10200 (58.824 sec)
I0823 00:52:11.907836 140559935440768 basic_session_run_hooks.py:260] lr = 0.00026377063 (58.824 sec)
I0823 00:53:06.864470 140559935440768 basic_session_run_hooks.py:692] global_step/sec: 1.81933
I0823 00:53:06.871208 140559935440768 basic_session_run_hooks.py:260] loss = 2.6168084, step = 10

...

Reading ../data/test.txt


I0823 07:07:00.308880 140559935440768 evaluation.py:275] Finished evaluation at 2019-08-23-07:07:00
I0823 07:07:00.310872 140559935440768 estimator.py:2039] Saving dict for global step 50000: global_step = 50000, loss = 2.3372955
I0823 07:07:00.322015 140559935440768 estimator.py:2099] Saving 'checkpoint_path' summary for global step 50000: ../model/gru_seq2seq/model.ckpt-50000
I0823 07:07:00.325476 140559935440768 interactiveshell.py:2882] Best Loss: 0.000
I0823 07:07:00.408036 140559935440768 estimator.py:1145] Calling model_fn.


[<tf.Variable 'Embedding/embedding:0' shape=(4718, 512) dtype=float32_ref>,
 <tf.Variable 'Encoder/bidirectional_rnn/fw/gru_cell/gates/kernel:0' shape=(1024, 1024) dtype=float32_ref>,
 <tf.Variable 'Encoder/bidirectional_rnn/fw/gru_cell/gates/bias:0' shape=(1024,) dtype=float32_ref>,
 <tf.Variable 'Encoder/bidirectional_rnn/fw/gru_cell/candidate/kernel:0' shape=(1024, 512) dtype=float32_ref>,
 <tf.Variable 'Encoder/bidirectional_rnn/fw/gru_cell/candidate/bias:0' shape=(512,) dtype=float32_ref>,
 <tf.Variable 'Encoder/bidirectional_rnn/bw/gru_cell/gates/kernel:0' shape=(1024, 1024) dtype=float32_ref>,
 <tf.Variable 'Encoder/bidirectional_rnn/bw/gru_cell/gates/bias:0' shape=(1024,) dtype=float32_ref>,
 <tf.Variable 'Encoder/bidirectional_rnn/bw/gru_cell/candidate/kernel:0' shape=(1024, 512) dtype=float32_ref>,
 <tf.Variable 'Encoder/bidirectional_rnn/bw/gru_cell/candidate/bias:0' shape=(512,) dtype=float32_ref>,
 <tf.Variable 'Encoder/dense/kernel:0' shape=(1024, 512) dtype=float32_ref>,

I0823 07:07:03.934907 140559935440768 estimator.py:1147] Done calling model_fn.
I0823 07:07:03.938937 140559935440768 basic_session_run_hooks.py:541] Create CheckpointSaverHook.
I0823 07:07:04.238487 140559935440768 monitored_session.py:240] Graph was finalized.
I0823 07:07:04.256716 140559935440768 saver.py:1280] Restoring parameters from ../model/gru_seq2seq/model.ckpt-50000


KeyboardInterrupt: ignored